In [1]:
from keras.applications import VGG16
from keras.optimizers import Adam
import numpy as np


Using TensorFlow backend.


In [2]:
img_rows = 224
img_cols = 224 

base_model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))


In [3]:
base_model.layers[0].__class__.__name__
base_model.layers[0].input

#Freezing all the layers by making their trainable=False
for layer in base_model.layers:
    layer.trainable=False
    
#Checking this
base_model.layers[12].trainable



False

In [4]:
for (i,layer) in enumerate(base_model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

    

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [5]:
from keras.layers import Dense, Flatten
from keras.models import Sequential

top_model = base_model.output
top_model = Flatten()(top_model)
top_model = Dense(512, activation='relu')(top_model)   #First added FCL dense layer
top_model = Dense(512, activation='relu')(top_model)    #Second added FCL dense layer
top_model = Dense(256, activation='relu')(top_model)    #Third added FCL dense layer
top_model = Dense(4, activation='softmax')(top_model)    #Output layer with 2 class labels


In [6]:
from keras.models import Model
newmodel = Model(inputs=base_model.input, outputs=top_model)


In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_data="ruhi/train/"
validation_data="ruhi/test/"
#Data image augmentation

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data,
        target_size=(img_rows, img_cols),
        class_mode='categorical')
 
test_generator = test_datagen.flow_from_directory(
        validation_data,
        target_size=(img_rows, img_cols),
        class_mode='categorical',
        shuffle=False)


Found 266 images belonging to 4 classes.
Found 33 images belonging to 4 classes.


In [8]:
from keras.optimizers import RMSprop
newmodel.compile(optimizer = 'adam',
                 loss = 'categorical_crossentropy',
                 metrics =['accuracy']
                )


In [9]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("VGG16.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
newmodel.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 266
nb_validation_samples = 33

# We only train 5 EPOCHS 
epochs = 5
batch_size = 16

history = newmodel.fit_generator(
    train_generator,
    steps_per_epoch = 50 ,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = test_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/5
50/50 [==============================] - 529s 11s/step - loss: 1.5548 - accuracy: 0.4262 - val_loss: 1.8365 - val_accuracy: 0.4848

Epoch 00001: val_loss improved from inf to 1.83651, saving model to VGG16.h5
Epoch 2/5
50/50 [==============================] - 521s 10s/step - loss: 0.7530 - accuracy: 0.6669 - val_loss: 0.1779 - val_accuracy: 0.6970

Epoch 00002: val_loss improved from 1.83651 to 0.17789, saving model to VGG16.h5
Epoch 3/5
50/50 [==============================] - 517s 10s/step - loss: 0.5186 - accuracy: 0.8038 - val_loss: 0.3192 - val_accuracy: 0.6970

Epoch 00003: val_loss did not improve from 0.17789
Epoch 4/5
50/50 [==============================] - 534s 11s/step - loss: 0.4781 - accuracy: 0.8094 - val_loss: 1.4270 - val_accuracy: 0.7273

Epoch 00004: val_loss did not improve from 0.17789
Epoch 5/5
50/50 [==============================] - 530s 11s/step - loss: 0.4276 - accuracy: 0.8228 - val_loss: 1.0046 - val_accuracy: 0.6364
Restoring model weights from th

In [10]:
newmodel.save('virat.h5')

In [11]:
from keras.models import load_model

classifier = load_model('virat.h5')


In [26]:
import os
import cv2
from os import listdir
from os.path import isfile, join

five_celeb_dict = {"[0]": "dhoni", 
                   "[1]": "hardik",
                   "[2]": "rohit",
                   "[3]": "virat"}

five_celeb_dict_n = {"dhoni": "dhoni", 
                     "hardik": "hardik",
                     "rohit": "rohit",
                     "virat": "virat"}

def draw_test(name, pred, im):
    celeb =five_celeb_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, celeb, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + five_celeb_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("ruhi/test/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Class - rohit
Class - rohit
Class - virat
Class - dhoni
Class - dhoni
Class - dhoni
Class - rohit
Class - dhoni
Class - dhoni
Class - hardik
